In [ ]:
import numpy as np
import sympy as sy
import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension()

<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">The Discrete Fourier Transform</strong></div>

https://www.youtube.com/watch?v=M0Sa8fLOajA&list=PL49CF3715CB9EF31D&index=27

# 1. The Discrete Fourier Basis

In [ ]:
def define_w(n):
    '''n^th root of 1'''
    return sy.exp(2j*sy.pi/n)

def FourierMatrix(n):
    '''Entry F_ij = w ^ (i j)'''
    w = define_w(n)
    l = sy.Matrix( [[ (w**i)**j for j in range(n)] for i in range(n)] )
    return l

def numFourierMatrix(n):
    # compute the numerical representation with the 1/sqrt(n) scaling included
    w = np.cos(2*np.pi/n)+1j*np.sin(2*np.pi/n)  # convert to floating point
    l = np.array( [[ w**(p*k) for k in range(n)] for p in range(n)] )*1/np.sqrt(n)
    return l

F = FourierMatrix(4)   # multiply by 1/sqrt(n) to make the matrix unitary. See below
F

<div style="float:left;width:65%;">
The Fourier Matrix $F$ of size $n \times n$ is an invertible matrix defined<br>$\quad$  from a <strong>root of unity</strong> $w = e^{i \frac{2 \pi}{n}}$:<br>
$\qquad \boxed{F = w^{i j}}\;\; i=0,1,\dots (n-1),\;\; j=0,1,\dots (n-1).$
</div>
<div style="float:left;width:25%;padding-left:1cm;">
<img src="../Figs/3rd_roots_of_unity.svg" width=100>
</div>

In [ ]:
FFh = F*F.T.conjugate()                  # F times its Hermitian Transpose
FFh.applyfunc(lambda x: sy.N(x,3))\
   .applyfunc( lambda x: 0 if abs(x) < 1e-10 else x ) # no surprise: we just need to scale M or its hermitian transpose
                                         # by convention, we scale both F and F^h by 1/sqrt(n)

The Fourier Matrix $F$ is invertible. since $F F^H = N$, the size of the matrix, the inverse of $F$ is given by<br>
$\quad F^{-1} = \frac{1}{N} F^H$.

A good choice of a scale factor for $F$ is to redefine the Fourier matrix by scaling it by $\frac{1}{\sqrt{N}}$. Then<br>
$\quad F \leftarrow \frac{1}{\sqrt{N}} F$, so that $F^{-1} = F^H$.

# 2. Example: Sines Sampled at N=256 Values

## 2.1 Example 1: A Sine Function

Let's set up a function of time sampled at $N = 256$ points

In [ ]:
N  = 256
F  = numFourierMatrix(N)
Fh = F.T.conj()

# ------------------------------------------------------------
t = np.linspace(0, 1, N)
x = 2*np.sin(2*np.pi*10*t)   # period of the sine is f=1/10 secs

pn.Row(hv.Curve((t,x), "t", "x")\
         .opts(title="Function  y = 2 sin( 2pi f t)", height=200, width=500) *\
       hv.Scatter((t,x))
      )

#### **Change of Basis**

If we think of $x$ as a vector, then $y = F x\;$ rewrites $x$<br>$\quad$ as a linear combination of the columns of $F^{-1}:\quad x = F^{-1} y$

$\quad$ This is just a **change of basis!**

##### **The Basis Vectors**

What are the basis vectors? Since the columns of $F$ are complex,<br>
$\quad$ we will **plot both the real and imaginary parts of each column**<br>
$\quad$ versus the index of the column entry:

In [ ]:
def plot( N, basis_vector_index, both=False ):
    h = hv.Curve(np.real(Fh[:,basis_vector_index]), "t", "x",   label='real part')*\
        hv.Curve(np.imag(Fh[:,basis_vector_index]),   label='imag part')

    if both and (basis_vector_index != 0): # index 0 would just repeat...
        h = h * \
        hv.Curve(np.real(Fh[:,N-basis_vector_index]), label='real part(N-i)')*\
        hv.Curve(np.imag(Fh[:,N-basis_vector_index]), label='imag part(N-i)')
        title = f'Basis Vectors # {basis_vector_index} and {N-basis_vector_index}'
    else:
        title=f'Basis Vector # {basis_vector_index}'

    return h.opts( title=title, width=650, legend_position='right')\
            .opts("Curve", muted_alpha=0)
pn.Row(pn.interact( lambda both, basis_vector_index: plot(N, basis_vector_index, both),
             basis_vector_index=(0,N//2), both=pn.widgets.Checkbox(name=" show both i and N-i components")))

So the basis vectors are sines and cosines with increasing frequencies!

##### **The Coordinate Vector of the Function**

The x vector we computed above was one such sine.<br>
$\quad$ Let's compute the corresponding y vector

The entries in this vectors are the complex multipliers of the columns of $F$.<br>
$\quad$ We are interested in their magnitude

In [ ]:
n = np.arange(N)  # the index of the basis vector
y = F @ x
pn.Row(hv.Spikes((n,abs(y)),"n", "y", label="magnitude of the component vector y").opts(width=650))

The reason we got two peaks is that we used a real vector x,<br>
$\quad$ i.e., not really a basis vector, but the difference of two basis vectors:<br>
$\quad$ the columns of $F$ at index 10, and index 256-10 (counting from 0)

Note there is some bleeding of the actual component to neighboring basis vectors<br>
$\quad$ due to inexact numerical computations.

**Remark:** drawing these plots as curves is misleading: we have a **discrete set of values**

## 2.2 Example 2: Sine and a Cosine, Different Frequencies

In [ ]:
# What if we make the function more complicated?
x = 2.0*np.sin(2*np.pi*10*t) \
   +0.5*np.cos(2*np.pi*100*t)
y = F @ x
pn.Row( hv.Spikes((t,x), "t", "x", label='x').opts(show_legend=False)*\
        hv.Curve( (t,x), "t", "x", label='x').opts(show_legend=False),
        hv.Spikes((n,abs(y)),"index", "abs_y", label="|y|"))

So we created an x vector composed of a sine with a superimposed wiggle with frequencies 10 and 100 Hz,<br>
and therefore, there are 4 entries in the transformed coefficient vector:<br> entries at 10 and 100, as well as 256-10, 256-100

# 3. A Function of Time

## 3.1 Sample a Function $x(t)$

In [ ]:
# What if we make the function more complicated?
x = 2.0*(0.5-t)**3 + 0.1*t \
   + 0.01*np.cos(2*np.pi*100*t)  # add a small, fast wiggle to the cubic
y = F @ x
pn.Row( hv.Curve( (t,x), "t", "x", label='x').opts(show_legend=False),
        hv.Spikes((n,abs(y)),"index", "abs_y", label="|y|"))

There are a lot of small entries In addition to the peaks at n=100, 256-n for the wiggle!<br>
This is easier to see on a log scale 

In [ ]:
y_abs = abs(y)
y_min =  min(y_abs)
y_max =  max(y_abs)
print("Smallest coefficient is", y_min )
pn.Row((hv.Curve((n,y_abs), "index", "abs_y")\
         .opts(logy=True,ylim=(y_min,1.01*y_max), width=600,muted_alpha=0,yticks=5,line_width=0.5,
               title = "coefficient vector abs.(y)")*\
        hv.Spikes((n,y_abs-y_min), "index", "abs_y").opts(logy=True,ylim=(y_min,1.01*y_max), position=y_min))
      )

What if we **throw out some of those small values,** (they are just small wiggles),<br>
and transform back
to our original basis? Is $\tilde{x} \approx F y$?

## 3.2 Removing Some Fourier Coefficients

In [ ]:
def ditch_values( y, smallest ):
    y_approx = np.array( [ v if abs(v) > smallest else 0 for v in y] )
    return y_approx

def ditch_values_plot( y, smallest ):
    y_approx = ditch_values( y, smallest )
    h        = hv.Spikes( (n,abs(y_approx)), "index", "abs_y", label='larger values only' ) *\
               hv.Curve( (n, abs(y)),        label='all values' ).opts(line_width=0.5)
    return h.opts( legend_position = 'right', width=700)

pn.Row(ditch_values_plot(y, 0.05))   # 0.05 includes the fast wiggle!

In [ ]:
def recovered_plot(y,  smallest ):
    y_approx = ditch_values( y, smallest )
    x_approx = Fh @ y_approx

    h_error = hv.Curve( (t, np.abs(x_approx - x)), "t", "x - x_recovered" ).opts(title="Error")
    h       = hv.Curve( (t,np.real(x)), "t", "x", label='original').opts( width=500, title="Original and Recovered Curve" ) *\
              hv.Curve( (t,np.real(x_approx)), "t", "x", label='approx')
    return h_error + h.opts(legend_position="right")
    
pn.Row( recovered_plot(y, 2e-3 ) )

In [ ]:
pn.interact( lambda smallest: recovered_plot( y, smallest).opts(title="Remove smallest Fourier Coefficients"),
             smallest=pn.widgets.FloatSlider(name='smallest',
                        start=y_min, end=y_min+0.2*(y_max-y_min), step=0.2*(y_max-y_min)/100) )

Click on the magnifying glass to zoom

**Remark:** The expansion forces the extreme y values to be the same; hence the large errors near the endpoints of the time interval

# 4. Take Away

The change of basis to Fourier Coefficients **reveals features** of the original series<br>
$\quad$ that were not readily apparent.

We can analyze the function in either representation, and introduce transformations<br>
$\quad$ in one domain that may be much harder in the other.

Note the clipping function *ditch_values()* we applied is **non-linear**

The structure of the $F$ matrix can be exploited to greatly **speed up the computation** of $F x$<br>
$\quad$ (the **Fast Fourier Transform**)